In [7]:
import pandas as pd 
import ast
import re
import numpy as np
import nltk
from textblob import TextBlob
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.util import ngrams
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
import pyLDAvis.gensim_models
# en_core_web_md = spacy.load("en_core_web_sm")
import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

c:\Users\bandy\AppData\Local\Programs\Python\Python310\lib\site-packages\gensim\matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bandy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bandy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bandy\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\bandy\AppData\Roaming\nltk_data...


True

In [8]:
df = pd.read_csv("./data/kickstarter_train.csv")

In [9]:
# convert 'video' to a binary categorical variable
df['video'].value_counts()
df['has_video'] = df['video'].apply(lambda x: 0 if pd.isnull(x) else 1)
df['has_video'].value_counts()

1    1015
0     542
Name: has_video, dtype: int64

## NLP features

In [10]:
# Text cleaning for: rewards, description, description story, description risks

def clean_text(df):
    def process_rewards(corpus):
    
        corpus_processed = []
        for row in corpus:
            row_processed = ""
            row = row.replace("\\n", " ")
            row = ast.literal_eval(row)

            for dict in row:
                row_processed += dict['rewards'].lower() + ' '
            
            
            row_processed = row_processed.replace("//",'')
            row_processed = re.sub(r'[^\w\s]', '', row_processed) # remove punctuation
            corpus_processed.append(row_processed)

        return corpus_processed
    
    def process_description_story(corpus):
        corpus_processed = []
        for row in corpus:
            row = str(row)
            row_processed = row.replace("\r", " " )
            row_processed = row_processed.replace("\n", " " )
            row_processed = re.sub(r'[^\w\s]', '', row_processed) # remove punctuation
            corpus_processed.append(row_processed if not pd.isnull(row_processed) else "") # handle NA

        return corpus_processed

    df["rewards_processed"] = process_rewards(df["rewards"])
    df["description_processed"] = process_description_story(df["description"])
    df["description_story_processed"] = process_description_story(df["description_story"])
    df["description_risks_processed"] = process_description_story(df["description_risks"])

    return df


In [11]:
class LemmatizeTokenizer(object):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
    def __call__(self, text):
        return [self.lemmatizer.lemmatize(word) for word in word_tokenize(text)]

In [12]:
def generate_nlp_features(df):
    
    # Rewards

    vect_rewards = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )

    rewards_processed = pd.Series(df["rewards_processed"])
    tfidf_fit_rewards = vect_rewards.fit(rewards_processed)
    rewards_tfidf_array = tfidf_fit_rewards.transform(rewards_processed).toarray()
    rewards_tfidf_df = pd.DataFrame(rewards_tfidf_array)
    rewards_tfidf_df.columns = list(map(lambda x : "rewards_" + str(x), rewards_tfidf_df.columns))
    df = pd.merge(df, rewards_tfidf_df , left_index=True, right_index=True)
    

    # Description

    vect_description = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )

    description = pd.Series(df["description_processed"])
    tfidf_fit_description = vect_description.fit(description)
    description_tfidf_array = tfidf_fit_description.transform(description).toarray()
    description_tfidf_df = pd.DataFrame(description_tfidf_array)
    description_tfidf_df.columns = list(map(lambda x : "description_" + str(x), description_tfidf_df.columns))
    df = pd.merge(df, description_tfidf_df , left_index=True, right_index=True)


    # Description Story

    vect_description_story = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )
    
    description_story_processed = pd.Series(df["description_story_processed"])
    tfidf_fit_description_story_processed = vect_description_story.fit(description_story_processed)
    description_story_processed_tfidf_array = tfidf_fit_description_story_processed.transform(description_story_processed).toarray()
    description_story_tfidf_df = pd.DataFrame(description_story_processed_tfidf_array)
    description_story_tfidf_df.columns = list(map(lambda x : "description_story_" + str(x), description_story_tfidf_df.columns))
    df = pd.merge(df, description_story_tfidf_df , left_index=True, right_index=True)


    # Description Risks

    vect_description_risks = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )

    description_risks_processed = pd.Series(df["description_risks_processed"])
    tfidf_fit_description_risks_processed = vect_description_risks.fit(description_risks_processed)
    description_risks_processed_tfidf_array = tfidf_fit_description_risks_processed.transform(description_risks_processed).toarray()
    description_risks_tfidf_df = pd.DataFrame(description_risks_processed_tfidf_array)
    description_risks_tfidf_df.columns = list(map(lambda x : "description_risks_" + str(x), description_risks_tfidf_df.columns))
    df = pd.merge(df, description_risks_tfidf_df , left_index=True, right_index=True)

    return df

In [ ]:
# test
#df = generate_nlp_features(clean_text(df))

In [13]:
for x in df.columns:
    print(x)

id
name
description
description_story
description_risks
rewards
category
pledged
goal
deadline
location
state
faq_count
update_count
backers_count
spotlight
staff_pick
video
launched_at
has_video


## Rewards Features

<font color="red"><strong>I dont think list can be a input to ML</strong></font>

In [14]:
def create_rewards_tiers(df):
    df["reward_tiers"] = df["rewards"].apply(lambda x : len(ast.literal_eval(x)))
    df = move_reward_tiers(df)
    return df

def create_all_reward_amount(df):
    df["min_reward"] = 0
    df["max_reward"] = 0

    for i in range(len(df)):
        all_reward_amount = []
        dict_list = ast.literal_eval(df.iloc[i, 5]) # Converts rewards column into dictionary
        for dict in dict_list:
            values_string = str(dict.values())
            reward_title = re.search(r"Pledge S\$ \d{1,3}(,\d{1,3})? or more", values_string) # Search for all reward titles

            if reward_title is not None:
                reward_amount = re.search(r"\d{1,3}(,\d{1,3})?", reward_title.group()) # Search for only the digits in reward amount
                if reward_amount is not None:
                    integer_amount = re.sub("[^\d\.]", "", reward_amount.group())
                    all_reward_amount.append(int(integer_amount))
            else:
                all_reward_amount.append(0) # If no reward title is found, add 0
        if len(all_reward_amount) > 0:
            df["min_reward"][i] = min(all_reward_amount)
            df["max_reward"][i] = max(all_reward_amount)
    df = move_min_max_reward(df)
    return df

# Rearange reward_tiers column to the right of rewards
def move_reward_tiers(df):
    cols = df.columns.tolist()
    cols = cols[:6] + [cols[-1]] + cols[6:-1]
    df = df[cols]
    return df

# Rearange all_reward_amount column to the right of reward_tiers
def move_min_max_reward(df):
    cols = df.columns.tolist()
    cols = cols[:7] + cols[-2:] + cols[7:-1]
    df = df[cols]
    return df

<>:20: DeprecationWarning: invalid escape sequence '\d'
<>:20: DeprecationWarning: invalid escape sequence '\d'
C:\Users\bandy\AppData\Local\Temp\ipykernel_5900\2405369866.py:20: DeprecationWarning: invalid escape sequence '\d'
  integer_amount = re.sub("[^\d\.]", "", reward_amount.group())


In [ ]:
'''
If lowest reward amount is not 0, the project is either already fully funded or cancelled.
Rewards should be sorted in ascending order, any amount to the right and less than the max means reward is no longer available.
'''

In [15]:
'''
Polarity is float which lies in the range of [-1,1] where 1 means positive statement and -1 means a negative statement. Subjective sentences generally refer to personal opinion, emotion or judgment whereas objective refers to factual information. Subjectivity is also a float which lies in the range of [0,1].
'''
def generate_sentiment_features(df):
    df = df.dropna(subset=['description_story_processed', 'description_risks_processed', 'description_processed', 'rewards_processed']) # NOTE: put at top with other dropnas from other features?
    df["description_story_polarity"] = df["description_story_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["description_story_subjectivity"] = df["description_story_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)
    df["description_polarity"] = df["description_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["description_subjectivity"] = df["description_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)
    df["description_risks_polarity"] = df["description_risks_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["description_risks_subjectivity"] = df["description_risks_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)
    df["rewards_polarity"] = df["rewards_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["rewards_subjectivity"] = df["rewards_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)          
    return df

In [16]:
def generate_word_count_features(df): # omitted description due to word limit, word count likely similar for all projects
    df['description_story_word_count'] = df["description_story_processed"].apply(lambda x: len(str(x).split(" ")))
    df['description_risks_word_count'] = df["description_risks_processed"].apply(lambda x: len(str(x).split(" ")))
    df['rewards_word_count'] = df["rewards_processed"].apply(lambda x: len(str(x).split(" ")))
    return df

## One-hot Encoding of Categorical Variables

when to drop one of the OHE columns: 
https://stats.stackexchange.com/questions/231285/dropping-one-of-the-columns-when-using-one-hot-encoding

In [17]:
def ohe_transform(enc, col, df):
    transformed = enc.transform(df[col].to_numpy().reshape(-1, 1))
    #Create a Pandas DataFrame of the hot encoded column
    ohe_df = pd.DataFrame(transformed, columns=enc.get_feature_names_out())
    #concat with original data
    data = pd.concat([df, ohe_df], axis=1).drop([col], axis=1)
    return data

def ohe_fit(col, df):
    enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
    enc.fit(df[col].unique().reshape(-1, 1))
    return (ohe_transform(enc, col, df), enc)

hold out on topic modelling first because it is unsupervised algorithm

In [ ]:
# df = clean_text(df)
# #generate BOW on description_story 
# #Our spaCy model:
# nlp = en_core_web_md.load()
# # Tags I want to remove from the text
# removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']
# tokens = []

# for story in nlp.pipe(df['description_story_processed']):
#    proj_tok = [token.lemma_.lower() for token in story if token.pos_ not in removal and not token.is_stop and token.is_alpha]
#    tokens.append(proj_tok)
    
# dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)
# # No_below: Tokens that appear in less than 5 documents are filtered out.
# # No_above: Tokens that appear in more than 50% of the total corpus are also removed as default.
# # Keep_n: We limit ourselves to the top 1000 most frequent tokens (default is 100.000). Set to ‘None’ if you want to keep all.

# df['story_tokens'] = tokens
# print(len(df))
# print(len(tokens))

# df['story_tokens']
# dictionary = Dictionary(df['story_tokens'])

# corpus = [dictionary.doc2bow(doc) for doc in df['story_tokens']]

# lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=50, num_topics=10, workers = 4, passes=10)

## Combine all feature generating functions

In [20]:
result = clean_text(df)
result = generate_nlp_features(result)
result = create_rewards_tiers(result)
result = create_all_reward_amount(result)
result = generate_sentiment_features(result)
result = generate_word_count_features(result)
result, category_encoder = ohe_fit('category', result) #use encoder to fit train data
result, location_encoder = ohe_fit('location', result) #use encoder to fit train data

c:\Users\bandy\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\bandy\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'doe', 'ha', 'might', 'must', "n't", 'need', 'sha', 'wa', 'wo', 'would'] not in stop_words.
  warnings.warn(
c:\Users\bandy\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\bandy\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your prep

UnboundLocalError: local variable 'df_copy' referenced before assignment

## Apply same feature engineering on Test Data

In [ ]:
test

,id,name,description,description_story,description_risks,rewards,reward_tiers,all_reward_amount,pledged,goal,...,x0_technology/web,x0_theater/comedy,x0_theater/experimental,x0_theater/immersive,x0_theater/musical,x0_theater/plays,"x0_Orchard, Singapore","x0_Queenstown, Singapore","x0_Sembawang, Singapore","x0_Singapore, Singapore"
0,505555120,Vertica Green - Mission to make cities green!,Vertical Gardens set up at schools that are fu...,StoryThat’s me (on the left). Anything related...,The majority of the cost for the project is up...,[{'rewards': 'Pledge S$ 5 or more\n\nAbout US$...,3,"[5, 25, 200]",89.0,195000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1161363320,The Trojan,"A crossbreed of armies & animals, innocence & ...","StoryWhat's The Deal?\n""All warfare is based o...",Logistic is one of the main challenges in maki...,[{'rewards': 'Pledge S$ 2 or more\n\nAbout $2\...,11,"[2, 12, 23, 23, 30, 50, 70, 90, 25, 45, 120]",410.0,8000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,152317277,"Plain Sight, a 4 Dimension production",an orphan boy must choose between future he dr...,StoryPlain Sight is a 10 minute short film sto...,"Although we have the skills for each roles, di...",[{'rewards': 'Pledge S$ 5 or more\n\nAbout US$...,5,"[5, 10, 50, 100, 500]",102.0,1000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1862517692,No Brainer #1-4 : A Solo Comix Anthology,The 48-page 4th issue comes with 2 big stories...,Story\n\n\n\n\n\n\n\nWhat is a No-Brainer?\nNo...,The book is all done and ready for printing. I...,[{'rewards': 'Pledge S$ 3 or more\n\nAbout US$...,6,"[3, 8, 14, 18, 75, 75]",5145.0,4000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1960297971,Toast the Raccoon Cute Plushie Toys + Keychains,Chonky Raccoon named Toast! It's made into a h...,StoryMeet our chunky little bandit mask friend...,Challenges we faced are often lies on overall ...,[{'rewards': 'Pledge S$ 27 or more\n\nAbout $2...,9,"[27, 36, 54, 61, 108, 121, 215, 54, 81]",70514.0,18000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,2129055939,The Breakfast Project Singapore,Join a foodie (also a food technologist) who i...,"StoryIf you are a health freak like me, you mi...",1st stage:- Meeting the volume requirement. \n...,[{'rewards': 'Pledge AU$ 30 or more\n\nAbout $...,2,"[0, 0]",30.0,4000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
289,299460577,Jellyfish - A Short Film Set In Borneo,"Set in a beautiful remote fishing village, thi...","StoryBlossoming sexuality, teenage desire and ...",NaN,"[{'rewards': ""Pledge $10 or more\n\nAbout $10\...",8,"[0, 0, 0, 0, 0, 0, 0, 0]",9200.0,8000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
290,753139167,Double Bass Speaker by UB+,Explosive bass and clarity from a portable spe...,Story\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,As the world is swept by Covid19 pandemic and ...,[{'rewards': 'Pledge S$ 209 or more\n\nAbout U...,13,"[209, 246, 251, 279, 283, 323, 403, 419, 487, ...",34343.0,10000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
291,1922446531,Touge Shakai,A Touge Racing Simcade,Story\n\n\n\n\n\n\n\nTouge Shakai (Mountain Pa...,The amount of content will be limited based on...,[{'rewards': 'Pledge S$ 3 or more\n\nAbout US$...,6,"[3, 15, 30, 60, 100, 1,000]",7038.0,24000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Export

In [ ]:
import time 

result.to_csv(f'data/train/kickstarter_train_final_{time.strftime("%Y%m%d-%H%M%S")}.csv', index=False)
test.to_csv(f'data/test/kickstarter_test_final_{time.strftime("%Y%m%d-%H%M%S")}.csv', index=False)

In [ ]:
#since there could be more than one categories for each project, create new features to split the categories.
# df['new_category'] = df.category.str.split("/", expand=False)
# split_cat = pd.DataFrame(df['new_category'].tolist(), columns=['category1', 'category2'])
# #each project should at least have 1 category, 'category2' can be "None". 
# #'category1' being the main category for the project.
# df = pd.concat([df, split_cat], axis=1)
# df.head()